In [1]:
import json
import math
from scripts_stock.cfg.out_file_name import OutFileName
from scripts_stock.cfg.set_dir import ProjectDir
from scripts_stock.cfg.stock_list import *
from scripts_stock.utils.analysis.stock_stat_index import df_to_stock_df, stock_kdj
import os
from scripts_stock.utils.common import CommonScript
from scripts_stock.utils.string_process import StringProcess

import plotly.figure_factory as ff
import numpy as np
from scripts_stock.data_base.get_table_info import GetDataFromDB

============ import stock list:  stock_list   ================


In [25]:
from datetime import datetime
from dateutil.relativedelta import relativedelta


def date_str_add_month(date_string="2024-10-01",add_month=2):
    # 假设你有一个日期字符串
    #date_string = "2024-10-01"

    # 将日期字符串转换为datetime对象
    date_object = datetime.strptime(date_string, "%Y-%m-%d")

    # 使用relativedelta给日期加两个月
    new_date_object = date_object + relativedelta(months=+add_month)

    # 将新的datetime对象格式化回字符串（如果需要）
    new_date_string = new_date_object.strftime("%Y-%m-%d")
    return new_date_string

In [26]:
owner_date = "2024-09-30"
update_date = "2024-10-31"
input_table = "r_t_owner_dfcf_2018_onwards"
obs_date_end = date_str_add_month(update_date, 1)
obs_date_end

'2024-11-30'

In [13]:

sql_str = f"""
    select HOLDER_NAME,HOLDER_TYPE,count(distinct SECURITY_CODE) as cnt
    from 
    {input_table}
    where substr(END_DATE,1,10)='{owner_date}'  and 
    HOLDER_TYPE<>'个人' 
    group by HOLDER_NAME,HOLDER_TYPE
    order by cnt DESC
"""

conn = CommonScript.connect_to_db("test.db")
df1 = pd.read_sql_query(sql_str, conn)
df1.head(4)

,HOLDER_NAME,HOLDER_TYPE,cnt
0,香港中央结算有限公司,其它,2428
1,招商银行股份有限公司-南方中证1000交易型开放式指数证券投资基金,证券投资基金,669
2,高盛公司有限责任公司,QFII,420
3,中国农业银行股份有限公司-中证500交易型开放式指数证券投资基金,证券投资基金,417


In [14]:
sql_v1 = f"""
select 
t1.HOLDER_NAME,
stock_index,
stock_name,
t1.HOLDER_STATEE,
t2.total_stock_host,
end_date
from 
(
    select HOLDER_NAME,
    SECURITY_CODE as stock_index,
    SECURITY_NAME_ABBR as stock_name,
    HOLDER_NAME,
    HOLDER_STATEE,
    substr(END_DATE,1,10) as end_date
    from 
    {input_table} 
    where substr(END_DATE,1,10)='{owner_date}' and HOLDER_TYPE<>'个人' 
    and (HOLDER_STATEE='新进' or HOLDER_STATEE='加仓')
) t1
inner join 
(
    select HOLDER_NAME,count(distinct SECURITY_CODE) as total_stock_host
    from 
    {input_table}
    where substr(END_DATE,1,10)='{owner_date}'  and 
    HOLDER_TYPE<>'个人' and (HOLDER_STATEE='新进' or HOLDER_STATEE='加仓')
    group by HOLDER_NAME
    having total_stock_host>=5
) t2
on t1.HOLDER_NAME=t2.HOLDER_NAME
"""

In [15]:
df_raw = pd.read_sql_query(sql_v1, conn)
stock_list_in = df_raw["stock_index"].unique()
df_raw.sort_values("total_stock_host")
print(len(stock_list_in))

3698


In [27]:
def get_stock_close_slope(stock_list_in, update_date, obs_date_end):
    slope_list = []
    stock_list = [] 
    for stock_ind in stock_list_in:
        try:
            df1 = GetDataFromDB.get_fuquan_all_one_stock_df(
                int(stock_ind)).sort_values("date")
            df2 = df1[(df1["date"] >= update_date) &
                    (df1["date"] <= obs_date_end)]
            df3 = df2.reset_index(drop=True)
            slope, intercept = np.polyfit(
                df3.index, df3["close"]/df3["close"].max(), 1)
            slope_list.append(slope)
            stock_list.append(stock_ind)
        except:
            # print(int(stock_ind))
            pass
    df4 = pd.DataFrame([slope_list, stock_list]).T
    df4.columns = ['slope', 'stock_index']
    return df4 

df4 = get_stock_close_slope(stock_list_in,update_date, obs_date_end)

In [35]:
df5 = df_raw.merge(df4)
df6 = df5[["HOLDER_NAME", "slope"]].groupby(
    "HOLDER_NAME").mean().reset_index()
df7 = df5[["HOLDER_NAME", "stock_name"]].groupby(
    "HOLDER_NAME")['stock_name'].nunique().reset_index()
df8 = df6.merge(df7)
df9 = df8.sort_values("slope", ascending=False)
df9.tail(20)

,HOLDER_NAME,slope,stock_name
9,中国国际金融股份有限公司,-0.003307,1
40,全国社保基金一一五组合,-0.003564,1
39,全国社保基金一一二组合,-0.003636,1
29,中国建设银行股份有限公司-易方达国防军工混合型证券投资基金,-0.003676,1
37,全国社保基金一一一组合,-0.003869,1
50,全国社保基金四一八组合,-0.004086,1
22,中国建设银行股份有限公司-中欧价值发现股票型证券投资基金,-0.004086,1
24,中国建设银行股份有限公司-中欧成长优选回报灵活配置混合型发起式证券投资基金,-0.004086,1
19,中国工商银行股份有限公司-南方中证全指房地产交易型开放式指数证券投资基金,-0.004311,5
27,中国建设银行股份有限公司-国泰中证军工交易型开放式指数证券投资基金,-0.004501,1


In [36]:
df5[df5["HOLDER_NAME"] == '全国社保基金六零二组合']

,HOLDER_NAME,stock_index,stock_name,HOLDER_STATEE,total_stock_host,end_date,slope
580,全国社保基金六零二组合,600383,金地集团,新进,5,2024-09-30,-0.006879


In [10]:
print(aa)

        slope stock_index
0     0.00135      002050
1   -0.000514      603899
2    0.000409      002271
3   -0.002161      601100
4    0.001888      002064
..        ...         ...
239  0.004035        2466
240 -0.000128        2555
241 -0.002299        2594
242  0.005743        2756
243 -0.002553        2938

[244 rows x 2 columns]


In [25]:
# stock slope 
slope_list = []
stock_list = []
for stock_ind in stock_list_in:
    try:
        df1 = GetDataFromDB.get_fuquan_all_one_stock_df(
            int(stock_ind)).sort_values("date")
        df2 = df1[(df1["date"] >= owner_date) &
                  (df1["date"] <= obs_date_end)]
        df3 = df2.reset_index(drop=True)
        slope, intercept = np.polyfit(
            df3.index, df3["close"]/df3["close"].max(), 1)
        slope_list.append(slope)
        stock_list.append(stock_ind)
    except:
        #print(int(stock_ind))
        pass

df4 = pd.DataFrame([slope_list, stock_list]).T
df4.columns = ['slope', 'stock_index']
print(df4.head(10))

10
626
638
678
705
707
826
880
923
965
1209
1229
1266
1269
1282
1300
1309
1316
1317
1336
1368
1896
2031
2076
2094
2103
2112
2172
2176
2229
2235
2272
2313
2316
2370
2388
2403
2455
2528
2553
2597
2607
2620
2647
2702
2712
2713
2723
2750
2762
2767
2786
2796
2805
2835
2886
2888
2889
2898
2899
2905
2910
2931
2981
2990
3007
3010
3016
3026
301329
430599
600171
600193
600203
600280
600287
600386
600505
600543
600580
600611
600624
600626
600650
600653
600676
600678
600727
600730
600753
600811
600843
600885
600979
601086
603004
603029
603042
603056
603097
603139
603155
603226
603266
603280
603344
603353
603378
603381
603533
603580
603598
603616
603628
603798
603818
603825
603829
603860
603887
603897
603955
603988
688075
688418
688709
831214
831741
834647
835550
836724
838107
900903
900914
900924
17
62
99
506
518
615
702
716
1236
1255
1288
1298
1387
2072
2084
2114
2177
2231
2339
2343
2377
2425
2518
2546
2584
2703
2729
2806
2903
2927
2952
2962
3025
200017
300016
300020
300043
300044
300067
300086
3

In [21]:
df1 = GetDataFromDB.get_fuquan_all_one_stock_df(
                    601398).sort_values("date")
df2 = df1[(df1["date"] >= '2024-09-30') & (df1["date"] <= '2024-11-30')]
df3 = df2.reset_index(drop=True)
slope, intercept = np.polyfit(
      df3.index, df3["close"]/df3["close"].max(), 1)
slope

-0.0005218195684512928

In [23]:
int(stock_ind)

10

In [31]:
slope_list = []
stock_list = []
for stock_ind in stock_list_in:
    try:
        df1 = GetDataFromDB.get_fuquan_all_one_stock_df(
            int(stock_ind)).sort_values("date")
        df2 = df1[(df1["date"] >= '2024-09-30') & (df1["date"] <= '2024-11-30')]
        df3 = df2.reset_index(drop=True)
        slope, intercept = np.polyfit(
            df3.index, df3["close"]/df3["close"].max(), 1)
        slope_list.append(slope)
        stock_list.append(stock_ind)
    except:
        pass

In [36]:
df4 = pd.DataFrame([slope_list, stock_list]).T
df4.columns = ['slope','stock_index']

In [54]:
df5 = df_raw.merge(df4)
df6 = df5[["HOLDER_NAME", "slope"]].groupby(
    "HOLDER_NAME").sum().reset_index()
df7 = df5[["HOLDER_NAME", "cnt"]].groupby("HOLDER_NAME").mean().reset_index()
df8 = df6.merge(df7)
df8.sort_values("slope", ascending=False).head(30)

,HOLDER_NAME,slope,cnt
72,香港中央结算有限公司,0.103051,1648.0
32,中国建设银行股份有限公司-易方达沪深300交易型开放式指数发起式证券投资基金,0.059252,142.0
17,中国工商银行股份有限公司-华泰柏瑞沪深300交易型开放式指数证券投资基金,0.058568,245.0
37,中国银行股份有限公司-嘉实沪深300交易型开放式指数证券投资基金,0.046449,30.0
15,中国工商银行股份有限公司-华夏沪深300交易型开放式指数证券投资基金,0.034,45.0
7,中国农业银行股份有限公司-中证500交易型开放式指数证券投资基金,0.01975,417.0
63,招商银行股份有限公司-泉果旭源三年持有期混合型证券投资基金,0.019069,9.0
27,中国建设银行股份有限公司-华夏能源革新股票型证券投资基金,0.014848,9.0
26,中国建设银行股份有限公司-华夏国证半导体芯片交易型开放式指数证券投资基金,0.014787,22.0
48,全国社保基金一零九组合,0.010452,8.0


In [ ]:
df_raw

In [59]:
from scripts_stock.utils.datetime.quater_end import *
quater_list = get_last_day_of_quarter_range()
a1 = [x for x in quater_list if '2021' in x or '2020' in x]
a1

['2021-12-31',
 '2021-09-30',
 '2021-06-30',
 '2021-03-31',
 '2020-12-31',
 '2020-09-30',
 '2020-06-30',
 '2020-03-31']